## Import

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import random
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
# faster rcnn model이 포함된 library
import torchvision
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
from torch.utils.data.sampler import WeightedRandomSampler

import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## Config

In [4]:
batch_size = 16
learning_rate = 5e-3
num_epochs = 60
seed = 42

In [5]:
seed_everything(seed)

## Dataset

In [6]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir):
        super().__init__()
        self.obj_label = []
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        #self.transforms = transforms
        for index in tqdm(range(len(self.coco.getImgIds()))):
            image_id = self.coco.getImgIds(imgIds=index)
            image_info = self.coco.loadImgs(image_id)[0]
            ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
            anns = self.coco.loadAnns(ann_ids)
            obj_list = np.array([x['category_id'] for x in anns])
            self.get_obj_label(obj_list)
    
    #   0          1             2        3        4       5     6         7            8          9         10
    # UNKNOWN, General trash, Paper, Paper pack, Metal, Glass, Plastic, Styrofoam, Plastic bag, Battery, Clothing
    #  160,            2782,    9311,     659,    562   , 610 ,  3090,     1343    ,  7643,       63,        177
    def get_obj_label(self,obj_list):
        if 9.0 in list(np.unique(obj_list)): self.obj_label.append(0)
        elif 0.0 in list(np.unique(obj_list)): self.obj_label.append(1)
        elif 10.0 in list(np.unique(obj_list)): self.obj_label.append(2)
        elif 4.0 in list(np.unique(obj_list)): self.obj_label.append(3)
        elif 5.0 in list(np.unique(obj_list)): self.obj_label.append(4)
        elif 3.0 in list(np.unique(obj_list)): self.obj_label.append(5)
        elif 7.0 in list(np.unique(obj_list)): self.obj_label.append(6)
        elif 1.0 in list(np.unique(obj_list)): self.obj_label.append(7)
        elif 6.0 in list(np.unique(obj_list)): self.obj_label.append(8)
        elif 8.0 in list(np.unique(obj_list)): self.obj_label.append(9)
        else: self.obj_label.append(10)

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image/255.

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        labels = np.array([x['category_id']+1 for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
                                
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)
        
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        return image, target, image_id, self.obj_label[index]
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [7]:
class MapTransform(Dataset):
    def __init__(self, dataset, transforms, test_mode):
        self.dataset = dataset
        self.transforms=transforms
        self.test_mode = test_mode
        self.obj_label=[]
        if test_mode==False:
            print('\nTrain data Label Setting.........\n')
            for index in tqdm(range(len(self.dataset))):
                self.obj_label.append(self.dataset[index][3])

    def __getitem__(self, index):
        image = self.dataset[index][0]
        target = self.dataset[index][1]
        image_id = self.dataset[index][2]
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': target['labels']
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id

    def __len__(self):
        return len(self.dataset)

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.3),
        A.VerticalFlip(p=0.4),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [9]:
annotation = '../input/data/train_all.json'
data_dir = '../input/data'
train_all_dataset = CustomDataset(annotation, data_dir)

loading annotations into memory...


 29%|██▉       | 958/3272 [00:00<00:00, 9577.09it/s]

Done (t=5.06s)
creating index...
index created!


100%|██████████| 3272/3272 [00:00<00:00, 8788.41it/s]


## Model Train

In [10]:
class CrossEntropyLabelSmooth(nn.Module):
    """Cross entropy loss with label smoothing regularizer.

    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.

    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes=12, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss

In [11]:
def fastrcnn_loss(class_logits, box_regression, labels, regression_targets):
    # type: (Tensor, Tensor, List[Tensor], List[Tensor]) -> Tuple[Tensor, Tensor]
    """
    Computes the loss for Faster R-CNN.
    Arguments:
        class_logits (Tensor)
        box_regression (Tensor)
        labels (list[BoxList])
        regression_targets (Tensor)
    Returns:
        classification_loss (Tensor)
        box_loss (Tensor)
    """

    labels = torch.cat(labels, dim=0)
    regression_targets = torch.cat(regression_targets, dim=0)
    labal_smooth_loss = CrossEntropyLabelSmooth(num_classes=12)
    classification_loss = labal_smooth_loss(class_logits, labels)

    # get indices that correspond to the regression targets for
    # the corresponding ground truth labels, to be used with
    # advanced indexing
    sampled_pos_inds_subset = torch.nonzero(labels > 0).squeeze(1)
    labels_pos = labels[sampled_pos_inds_subset]
    N, num_classes = class_logits.shape
    box_regression = box_regression.reshape(N, -1, 4)

    box_loss = det_utils.smooth_l1_loss(
        box_regression[sampled_pos_inds_subset, labels_pos],
        regression_targets[sampled_pos_inds_subset],
        beta=1 / 9,
        size_average=False,
    )
    box_loss = box_loss / labels.numel()

    return classification_loss, box_loss

In [12]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.fastrcnn_loss = fastrcnn_loss
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [13]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [14]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0, path=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_mAP, model):
        score = val_mAP
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)

In [15]:
def train_fn(current_fold, model, train_data_loader, val_data_loader, optimizer, scheduler, device):
    coco = get_coco_api_from_dataset(val_data_loader.dataset)
    model.train()
    loss_hist = Averager()
    save_path = './saved/'+str(current_fold)+'_checkpoint.pth'
    early_stop = EarlyStopping(path=save_path)
    for epoch in range(num_epochs):
        loss_hist.reset()
        model.train()
        for images, targets, image_ids in tqdm(train_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = list(image.float().to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Mix up
            if random.random() > 0.5 and epoch>2: # warm up
                shuffle_indices = torch.randperm(len(images))
                indices = torch.arange(len(images))
                lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
                for i, si in zip(indices, shuffle_indices):
                    images[i] = lam * images[i] + (1-lam) * images[si]
                mix_targets = []
                for i, si in zip(indices, shuffle_indices):
                    if i.item()==si.item():
                        target = targets[i.item()]
                    else:
                        target = {
                            'boxes': torch.cat([targets[i.item()]['boxes'], targets[si.item()]['boxes']]),
                            'labels': torch.cat([targets[i.item()]['labels'], targets[si.item()]['labels']]),
                            'image_id': torch.cat([targets[i.item()]['image_id'], targets[si.item()]['image_id']]),
                            'area': torch.cat([targets[i.item()]['area'], targets[si.item()]['area']]),
                            'iscrowd': torch.cat([targets[i.item()]['iscrowd'], targets[si.item()]['iscrowd']])
                        }
                    mix_targets.append(target)
                targets=mix_targets

            # calculate loss
            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()

            loss_hist.send(loss_value)

            # backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
        
        # Evaluation
        val_mAP = vali_fn(coco, val_data_loader,model,device)
        # Scheduler Step
        scheduler.step(val_mAP)
        
        print(f"Epoch #{epoch+1} train loss: {loss_hist.value} vali mAP: {val_mAP}")
        early_stop(val_mAP,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [16]:
def vali_fn(coco, val_data_loader, model, device):
    model.eval()
    iou_types = ['bbox']
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = list(image.float().to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)

        outputs = [{k: v.to(device) for k, v in t.items()} for t in outputs]

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        
        coco_evaluator.update(res)
    
    coco_evaluator.synchronize_between_processes()
    coco_evaluator.accumulate()
    mAP=coco_evaluator.summarize()
    model.train()
    # CV mAP50 : Area = ALL
    return mAP.stats[1].item()

In [17]:
folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
#folds=KFold(n_splits=5,shuffle=True,random_state=seed)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(train_all_dataset,train_all_dataset.obj_label)):
#for current_fold,(train_idx, vali_idx) in enumerate(folds.split(train_all_dataset)):
    print("-"*80)
    print(f'Current Fold : [{current_fold}]')
    print("-"*80)
    # K-fold Dataset
    train_data=torch.utils.data.Subset(train_all_dataset,train_idx)
    vali_data=torch.utils.data.Subset(train_all_dataset,vali_idx)

    trans_train_data=MapTransform(train_data,get_train_transform(),False)
    trans_vali_data=MapTransform(vali_data,get_valid_transform(),True)
    
    counts = np.bincount(trans_train_data.obj_label)
    labels_weights = 1. / counts
    weights = labels_weights[trans_train_data.obj_label]
    weight_sampler=WeightedRandomSampler(weights, len(weights))

    train_data_loader=DataLoader(trans_train_data,batch_size=batch_size,sampler=weight_sampler,collate_fn=collate_fn,num_workers=0)
    val_data_loader=DataLoader(trans_vali_data,batch_size=1,shuffle=False,collate_fn=collate_fn,num_workers=0)
    # Model
    model = get_model(12)
    model.to(device)
    model.eval()
    # setting
    #optimizer = torch.optim.Adam(params,lr=learning_rate)
    #optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=0.0001)
    #optimizer = torch.optim.AdamW(params,lr=3e-4,weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
    # Run
    train_fn(current_fold, model, train_data_loader, val_data_loader, optimizer, scheduler, device)

  1%|          | 14/2617 [00:00<00:19, 136.50it/s]

--------------------------------------------------------------------------------
Current Fold : [0]
--------------------------------------------------------------------------------

Train data Label Setting.........



  0%|          | 0/164 [00:00<?, ?it/s]

creating index...
index created!


100%|██████████| 655/655 [00:46<00:00, 14.20it/s]


...

Accumulating evaluation results...


  0%|          | 0/164 [00:00<?, ?it/s]

DONE (t=0.85s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.277
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.378
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.168
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.385
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.146
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.398
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.629
Epoch #

100%|██████████| 654/654 [00:45<00:00, 14.26it/s]


Accumulating evaluation results...
DONE (t=0.86s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.251
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.276
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.168
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.383
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.421
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.146
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la